In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

In [2]:
# 生データを引数として受け取り、そのデータの問題サイズの値に合わせた追加の初期値を追加し、引数として渡されたDFにデータが付与されたDFを返す関数
# 引数：生データDF
# 返り値：生データDFにベンチマークごとの初期値が付与されたDF


def addInitDataToRawDF(rawDF):
    return rawDF


def test_addInitDataToRawDF():
    # rawDF の作成（cg）
    functionName = [
        "functionAtCG",
    ]
    functionCallNum = [1]
    benchmarkName = ["cg"]
    benchmarkClass = ["A"]
    process = [2]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    # CG特有の初期変数を代入する
    rawDF["na"] = 14000
    rawDF["nonzer"] = 11
    rawDF["niter"] = 15
    rawDF["shift"] = "20.d0"
    # rawDF に手動で関数のやっていることを実施した DF を作成
    # 比較して正しいことを確認
    # 引数として渡したDFと比較して、異なっていることを確認
    
    # rawDF の作成（ep）
    functionName = [
        "functionAtEP",
    ]
    functionCallNum = [2]
    benchmarkName = ["ep"]
    benchmarkClass = ["B"]
    process = [4]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    # rawDF に手動で関数のやっていることを実施した DF を作成
    # 比較して正しいことを確認
    # 引数として渡したDFと比較して、異なっていることを確認
    
    # rawDF の作成（ft）
    functionName = [
        "functionAtFT",
    ]
    functionCallNum = [4]
    benchmarkName = ["ft"]
    benchmarkClass = ["C"]
    process = [8]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    rawDF["d1"] = 512
    rawDF["d2"] = 512
    rawDF["d3"] = 512
    rawDF["nt"] = 20
    # rawDF に手動で関数のやっていることを実施した DF を作成
    # 比較して正しいことを確認
    # 引数として渡したDFと比較して、異なっていることを確認
    
    # rawDF の作成（is）
    functionName = [
        "functionAtIS",
    ]
    functionCallNum = [8]
    benchmarkName = ["is"]
    benchmarkClass = ["D"]
    process = [16]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    rawDF["NUM_PROCS"] = 1
    rawDF["ONE"] = 1
    rawDF["TOTAL_KEYS_LOG_2"] = 29
    rawDF["MAX_KEY_LOG_2"] = 27
    rawDF["NUM_BUCKETS_LOG_2"] = 10
    rawDF["MIN_PROCS"] = 4
    # rawDF に手動で関数のやっていることを実施した DF を作成
    # 比較して正しいことを確認
    # 引数として渡したDFと比較して、異なっていることを確認
    
    # rawDF の作成（lu）
    functionName = [
        "functionAtLU",
    ]
    functionCallNum = [16]
    benchmarkName = ["lu"]
    benchmarkClass = ["E"]
    process = [32]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    # rawDF に手動で関数のやっていることを実施した DF を作成
    # 比較して正しいことを確認
    # 引数として渡したDFと比較して、異なっていることを確認
    
    # rawDF の作成（mg）
    functionName = [
        "functionAtMG",
    ]
    functionCallNum = [32]
    benchmarkName = ["mg"]
    benchmarkClass = ["F"]
    process = [64]
    rawDict = {
        "functionName": functionName,
        "functionCallNum": functionCallNum,
        "benchmarkName": benchmarkName,
        "benchmarkClass": benchmarkClass,
        "process": process,
    }
    rawDF = pd.DataFrame(rawDict)
    # rawDF に手動で関数のやっていることを実施した DF を作成
    # 比較して正しいことを確認
    # 引数として渡したDFと比較して、異なっていることを確認
    
    pass
    return rawDF


sampleRawDF = returnCollectedExistingData(
    benchmarkNames=["cg"],
    classes=["A", "B", "C", "D", "E", "F"],
    processes=[2, 4, 8, 16, 32, 64, 128, 256],
    csvDirPath=csvDirPath,
)
sampleRawDF

test_addInitDataToRawDF()

,functionName,functionCallNum,benchmarkName,benchmarkClass,process
0,functionAtMG,32,mg,F,64
